In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split

%matplotlib inline

In [2]:
dataset = datasets.MNIST('data', train = True, transform=transforms.ToTensor())

c:\users\admin\appdata\local\programs\python\python38\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [3]:
len(dataset)

60000

In [4]:
train_ds, val_ds = random_split(dataset, [50000,10000])

In [5]:
# import time
# pin_memory = True
# print('pin_memory is', pin_memory)
 
# for num_workers in range(0, 20, 1): 
#     tndl = torch.utils.data.DataLoader(train_ds, batch_size=128,
#     num_workers=num_workers, pin_memory=pin_memory)
#     start = time.time()
#     for epoch in range(1, 5):
#         for i, data in enumerate(train_loader):
#             pass
#     end = time.time()
#     print("Finish with:{} second, num_workers={}".format(end - start, num_workers))
 

In [6]:
train_loader = DataLoader(train_ds, batch_size=128, shuffle=True, num_workers=1, pin_memory=True)
val_loader = DataLoader(val_ds, batch_size=128, shuffle=True, num_workers=1, pin_memory=True)

In [7]:
28*28


784

In [8]:
t = torch.tensor([[[2,2],[5,4]],[[2,2],[5,4]]])

In [9]:
t.shape

torch.Size([2, 2, 2])

In [10]:
t = t.flatten()
t.shape

torch.Size([8])

# model

In [11]:
class MnistCNN(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(1, 8, 3,padding=1) #out will be b,8,28,28
        self.flatten = nn.Flatten() # out will be b,8*28*28
        self.linear = nn.Linear(8*28*28,10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.flatten(x)
        x.shape
        x = self.linear(x)
        return x
        
    def training_step(self,batch):
        images,labels = batch
        out = model(images)
        loss = F.cross_entropy(out,labels)
        return loss
        
    def validation_step(self,batch):
        images,labels = batch
        out = model(images)
        loss = F.cross_entropy(out,labels)
        acc = accuracy(out,labels)
        return {'validation_loss':loss.detach(), 'validation_accuracy':acc}
    
    @torch.no_grad()
    def evaluate(self, val_loader):
        val_out_list = [self.validation_step(batch) for batch in val_loader]
        batch_loss = [x['validation_loss'] for x in val_out_list]
        batch_acc = [x['validation_accuracy'] for x in val_out_list]
        epoch_loss = torch.stack(batch_loss).mean()
        epoch_acc = torch.stack(batch_acc).mean()
        result = { 'val_loss':epoch_loss, 'val_acc': epoch_acc}
        return result

    def epoch_end(self,epoch, result):
            print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
                epoch, result['train_loss'], result['val_loss'], result['val_acc']))

#     def validation_at_end_of_epoch(self, val_out_list):
#         batch_loss = x[loss] for x in val_out_list
#         batch_acc = x[acc] for x in val_out_list
#         epoch_loss = stack(batch_loss).mean
#         epoch_acc = stack(batch_acc).mean
#         return { 'val_loss':epoch_loss.item(), 'val_acc': epoch_acc.item()}

    #def result_at_end_of_epoch():
    


        
    
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

# training and evalution

In [12]:
# @torch.no_grad()
# def evaluate(model, val_loader):
#     val_out_list = [model.validation_step(batch) for batch in val_loader]
#     batch_loss = [x['validation_loss'] for x in val_out_list]
#     batch_acc = [x['validation_accuracy'] for x in val_out_list]
#     epoch_loss = torch.stack(batch_loss).mean()
#     epoch_acc = torch.stack(batch_acc).mean()
#     result = { 'val_loss':epoch_loss, 'val_acc': epoch_acc}
#     return result

# def epoch_end(epoch, result):
#         print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
#             epoch, result['train_loss'], result['val_loss'], result['val_acc']))

In [13]:
def fit(epochs,lr,model,train_loader,val_loader,opt_fn):
    history = []
    optimizer = opt_fn(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase 
        model.train()
        train_losses = []
        for batch in train_loader:
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = model.evaluate(val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        model.epoch_end(epoch, result)
        history.append(result)
    return history

In [14]:
batch_size=128
lr=0.001
epochs=5
opt_fn=torch.optim.SGD


In [15]:
model=MnistCNN()

In [16]:
%%time
torch.set_num_threads(4)

Wall time: 52.5 ms


In [17]:
%%time
fit(epochs,lr,model,train_loader,val_loader,opt_fn)

Epoch [0], train_loss: 2.1599, val_loss: 1.9750, val_acc: 0.6657
Epoch [1], train_loss: 1.6696, val_loss: 1.3119, val_acc: 0.7940
Epoch [2], train_loss: 1.0349, val_loss: 0.8127, val_acc: 0.8383
Epoch [3], train_loss: 0.7017, val_loss: 0.6066, val_acc: 0.8616
Epoch [4], train_loss: 0.5623, val_loss: 0.5121, val_acc: 0.8707
Wall time: 2min 11s


[{'val_loss': tensor(1.9750),
  'val_acc': tensor(0.6657),
  'train_loss': 2.159942150115967},
 {'val_loss': tensor(1.3119),
  'val_acc': tensor(0.7940),
  'train_loss': 1.669646143913269},
 {'val_loss': tensor(0.8127),
  'val_acc': tensor(0.8383),
  'train_loss': 1.034921646118164},
 {'val_loss': tensor(0.6066),
  'val_acc': tensor(0.8616),
  'train_loss': 0.7016560435295105},
 {'val_loss': tensor(0.5121),
  'val_acc': tensor(0.8707),
  'train_loss': 0.5623199343681335}]

In [18]:
%%time
fit(epochs,lr,model,train_loader,val_loader,opt_fn)

Epoch [0], train_loss: 0.4921, val_loss: 0.4665, val_acc: 0.8765
Epoch [1], train_loss: 0.4509, val_loss: 0.4331, val_acc: 0.8847
Epoch [2], train_loss: 0.4234, val_loss: 0.4102, val_acc: 0.8884
Epoch [3], train_loss: 0.4039, val_loss: 0.3887, val_acc: 0.8907
Epoch [4], train_loss: 0.3894, val_loss: 0.3755, val_acc: 0.8951
Wall time: 2min 10s


[{'val_loss': tensor(0.4665),
  'val_acc': tensor(0.8765),
  'train_loss': 0.4920504689216614},
 {'val_loss': tensor(0.4331),
  'val_acc': tensor(0.8847),
  'train_loss': 0.4509265720844269},
 {'val_loss': tensor(0.4102),
  'val_acc': tensor(0.8884),
  'train_loss': 0.4233623147010803},
 {'val_loss': tensor(0.3887),
  'val_acc': tensor(0.8907),
  'train_loss': 0.4038562476634979},
 {'val_loss': tensor(0.3755),
  'val_acc': tensor(0.8951),
  'train_loss': 0.38939595222473145}]

# best results are with 0 num_workers